# Sanity Checks

Interactive notebook for exploring twin prime selection bias computations.

This notebook is for exploration and debugging only. All paper results should be generated via `run_all.py`.

In [ ]:
import sys
sys.path.insert(0, '..')

import numpy as np
import matplotlib.pyplot as plt

from src.primes import primes_upto, prime_flags_upto
from src.sieve_pairs import pair_values, pair_state, compute_all_states, STATES
from src.factorization import spf_sieve, omega

## 1. Basic Prime Generation

In [ ]:
# Test prime generation
primes = primes_upto(100)
print(f"Primes up to 100: {primes}")
print(f"Count: {len(primes)}")

# Verify first few
assert list(primes[:10]) == [2, 3, 5, 7, 11, 13, 17, 19, 23, 29]

## 2. Pair States

In [ ]:
# Test pair classification
K = 10
N = 6 * K + 1
flags = prime_flags_upto(N)

print("First 10 pairs:")
for k in range(1, K + 1):
    a, b = pair_values(k)
    state = pair_state(a, b, flags)
    print(f"  k={k}: ({a}, {b}) -> {state}")

## 3. State Distribution

In [ ]:
# Check state frequencies for moderate K
K = 10000
N = 6 * K + 1
flags = prime_flags_upto(N)
states = compute_all_states(K, flags)

print(f"State distribution for K={K:,}:")
for state in STATES:
    count = np.sum(states == state)
    print(f"  {state}: {count:,} ({100*count/K:.2f}%)")

## 4. Factorization Check

In [ ]:
# Test omega function
N = 1000
spf = spf_sieve(N)

test_cases = [
    (1, 0),    # 1 has no prime factors
    (2, 1),    # 2 = 2
    (6, 2),    # 6 = 2 * 3
    (12, 2),   # 12 = 2^2 * 3
    (30, 3),   # 30 = 2 * 3 * 5
    (210, 4),  # 210 = 2 * 3 * 5 * 7
]

print("Omega function tests:")
for n, expected in test_cases:
    result = omega(n, spf)
    status = "OK" if result == expected else "FAIL"
    print(f"  omega({n}) = {result} (expected {expected}) [{status}]")

## 5. Transfer Matrix Sanity

In [ ]:
from src.transfer_matrix import T_p, initial_distribution, apply_primes

# Check that transfer matrices are stochastic (columns sum to 1)
for p in [5, 7, 11, 13]:
    T = T_p(p)
    col_sums = T.sum(axis=0)
    print(f"p={p}: column sums = {col_sums}")
    assert np.allclose(col_sums, 1.0), f"T_{p} is not stochastic!"

In [ ]:
# Check state evolution
from src.coefficient_extraction import state_probabilities

print("State probabilities vs prime cutoff:")
for P in [10, 50, 100, 200]:
    probs = state_probabilities(P)
    print(f"  P={P}: PP={probs['PP']:.4f}, PC={probs['PC']:.4f}, CP={probs['CP']:.4f}, CC={probs['CC']:.4f}")

## 6. Quick Visual Check

In [ ]:
# Plot omega distribution for PP pairs
K = 50000
N = 6 * K + 1
flags = prime_flags_upto(N)
spf = spf_sieve(N)
states = compute_all_states(K, flags)

pp_mask = states == 'PP'
pp_indices = np.where(pp_mask)[0]

omega_a = []
omega_b = []
for i in pp_indices:
    a, b = pair_values(i + 1)
    omega_a.append(omega(a, spf))
    omega_b.append(omega(b, spf))

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

ax1.hist(omega_a, bins=range(0, 10), density=True, alpha=0.7, label='6k-1 (prime)')
ax1.set_xlabel('omega')
ax1.set_ylabel('Density')
ax1.set_title('PP pairs: 6k-1 (always prime, omega=1)')

ax2.hist(omega_b, bins=range(0, 10), density=True, alpha=0.7, label='6k+1 (prime)')
ax2.set_xlabel('omega')
ax2.set_ylabel('Density')
ax2.set_title('PP pairs: 6k+1 (always prime, omega=1)')

plt.tight_layout()
plt.show()